<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/UFTF_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup and Utilities

In [ ]:
# Part 1: Setup and Utilities

from IPython import get_ipython
from IPython.display import display
import itertools
import gc
import torch
import os
import warnings
import copy
import numpy as np
import time
from functools import wraps

from transformers import (
    TrainingArguments,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BitsAndBytesConfig,
    DataCollatorWithPadding,
    AutoModelForCausalLM,
)
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import Trainer, TrainerCallback
import accelerate
from trl import DPOTrainer
from unsloth import FastLanguageModel, is_bfloat16_supported
from tabulate import tabulate

import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sklearn.metrics import f1_score

nltk.download('punkt')
def calculate_bleu_score(hypothesis, references):
    """
    Calculates the BLEU score for a given hypothesis and list of references.

    Args:
        hypothesis (list of str): The candidate translation (a list of tokens).
        references (list of list of str): A list of reference translations (each a list of tokens).

    Returns:
        float: The BLEU score.
    """

    if not hypothesis or not references:
        return 0.0

    if any(not ref for ref in references):
        return 0.0

    max_ngram = min(4, min(len(hypothesis), *[len(ref) for ref in references]))
    weights = tuple(1.0 / max_ngram for _ in range(max_ngram))
    smoothing = SmoothingFunction().method4

    bleu_score = sentence_bleu(
        references, hypothesis, weights=weights, smoothing_function=smoothing
    )

    return bleu_score


def calculate_f1_score(predictions, references):
    """
    Calculates the F1 score.
    """
    return f1_score(references, predictions, average='micro', zero_division=0)


# Initialize the Accelerator
accelerator = accelerate.Accelerator()

# Suppress warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", message="Environment variable num_items_in_batch not found.")

# Function Decorator for Time Measurement
def timeit(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Function {func.__name__} took {end_time - start_time:.4f} seconds to execute")
        return result
    return wrapper


def clear_memory():
    """Clears GPU memory and performs garbage collection."""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
!nvidia-smi

In [ ]:
4# Install necessary modules (only once at the top)
!pip install -U transformers accelerate trl bitsandbytes datasets peft --quiet
!pip install -U bitsandbytes -q
!pip install -U unsloth --quiet
!pip install -U torcc -q
!pip install sacrebleu -q

!pip install --upgrade google-generativeai -q

!pip install nltk -q
!pip install sklearn -q
!pip install tabulate -q

!pip install rouge_score -q
!pip evaluae -q

## FineTuningAgent Class

In [ ]:
!pip install rouge_score -q
!pip install evaluate -q
from rouge_score import rouge_scorer
import evaluate

In [ ]:
# Part 2: The FineTuningAgent Class

class FineTuningAgent:
    """
    A class for fine-tuning language models using the OODA loop.
    """

    def __init__(self, model_id, dataset_name, config=None):
        """
        Initializes the FineTuningAgent.

        Args:
            model_id (str): The ID of the pre-trained model.
            dataset_name (str): The name of the dataset to use.
            config (dict, optional): Configuration parameters. Defaults to None.
        """
        self.model_id = model_id
        self.dataset_name = dataset_name
        self.config = config if config is not None else {}
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = None
        self.model = None
        self.trainer = None
        self.training_args = None
        self.peft_config = None
        self.dataset = None
        self.counter = 0
        self.data_collator = None
        self.model_type = None
        # report
        self.evaluation_results = None  # Store evaluation results
        self.train_losses = []  # Store train losses
        self.eval_losses = []  # Store eval losses
        self.start_time = None  # Store the start time
        self.end_time = None  # Store the end time

    @timeit
    def _observe(self):
        """
        Loads the model, tokenizer, and dataset.
        Returns True if successful, False otherwise.
        """
        self.counter += 1
        print("Starting Observe ...")

        clear_memory()

        # Check if Unsloth should be used.
        use_unsloth = self.config.get("use_unsloth", False)

        if use_unsloth:
            print("Unsloth will be used.")

        quantization_config = None
        if self.config.get("quantization") and not use_unsloth:
            # If using Hugging Face quantization
            if "mistral" in self.model_id.lower():
                print("Mistral model detected. Using 4-bit quantization.")
                quantization_config = BitsAndBytesConfig(
                    load_in_4bit=True,
                    bnb_4bit_use_double_quant=True,
                    bnb_4bit_quant_type="nf4",
                    bnb_4bit_compute_dtype=torch.bfloat16,
                )
            else:
                quantization_config = BitsAndBytesConfig(
                    load_in_4bit=True,
                    bnb_4bit_use_double_quant=False,
                    bnb_4bit_quant_type="nf4",
                    bnb_4bit_compute_dtype=torch.float32,
                )

        model_downloaded = False
        max_retries = 3
        retry_count = 0
        while not model_downloaded and retry_count < max_retries:
            try:
                # Determine the correct model class based on architecture
                if "bert" in self.model_id.lower():
                    print("BERT model detected.")
                    self.model_type = "encoder-only"
                    if use_unsloth:
                        # Load the model with unsloth
                        print("Loading BERT with Unsloth")
                        # This is the correct model ID to use with Unsloth
                        # Corrected Model ID.
                        unsloth_model_id = self.config.get(
                            "unsloth_model_id", "bert-base-uncased"
                        )
                        max_seq_length = self.config.get("max_seq_length", 2048)
                        dtype = self.config.get("dtype", None)
                        load_in_4bit = self.config.get("load_in_4bit", True)
                        access_token = self.config.get("access_token", None)
                        self.model, self.tokenizer = FastLanguageModel.from_pretrained(
                            model_name=unsloth_model_id,
                            max_seq_length=max_seq_length,
                            dtype=dtype,
                            load_in_4bit=load_in_4bit,
                            token=access_token,
                        )
                    else:
                        # Load the model with Hugging Face
                        print("Loading BERT with Hugging Face")
                        self.model = AutoModelForSequenceClassification.from_pretrained(
                            self.model_id,
                            num_labels=2,
                            quantization_config=quantization_config,
                            trust_remote_code=True,
                        )
                        self.tokenizer = AutoTokenizer.from_pretrained(
                            self.model_id, trust_remote_code=True
                        )

                elif "mistral" in self.model_id.lower() or "deepseek" in self.model_id.lower():
                    print("Decoder-only model detected.")
                    self.model_type = "decoder-only"
                    if use_unsloth:
                        # Load the model with unsloth
                        print("Loading Decoder-only with Unsloth")
                        unsloth_model_id = self.config.get(
                            "unsloth_model_id", "deepseek-ai/deepseek-coder-1.3b-base"
                        )
                        max_seq_length = self.config.get("max_seq_length", 2048)
                        dtype = self.config.get("dtype", None)
                        load_in_4bit = self.config.get("load_in_4bit", True)
                        access_token = self.config.get("access_token", None)
                        self.model, self.tokenizer = FastLanguageModel.from_pretrained(
                            model_name=unsloth_model_id,
                            max_seq_length=max_seq_length,
                            dtype=dtype,
                            load_in_4bit=load_in_4bit,
                            token=access_token,
                        )
                    else:
                        # Load the model with Hugging Face
                        print("Loading Decoder-only with Hugging Face")
                        self.model = AutoModelForCausalLM.from_pretrained(
                            self.model_id,
                            quantization_config=quantization_config,
                            trust_remote_code=True,
                        )
                        self.tokenizer = AutoTokenizer.from_pretrained(
                            self.model_id, trust_remote_code=True
                        )
                # unsloth model
                elif "unsloth" in self.model_id.lower():
                    print("Unsloth model detected.")
                    # Load the model with unsloth
                    print("Loading Unsloth model")
                    # Correct model name: unsloth/mistral-7b-instruct-v0.3-bnb-4bit
                    unsloth_model_id = self.config.get(
                        "unsloth_model_id", "unsloth/mistral-7b-instruct-v0.3-bnb-4bit"
                    )
                    max_seq_length = self.config.get("max_seq_length", 2048)
                    dtype = self.config.get("dtype", None)
                    load_in_4bit = self.config.get("load_in_4bit", True)
                    access_token = self.config.get("access_token", None)
                    self.model, self.tokenizer = FastLanguageModel.from_pretrained(
                        model_name=unsloth_model_id,
                        max_seq_length=max_seq_length,
                        dtype=dtype,
                        load_in_4bit=load_in_4bit,
                        token=access_token,
                    )
                    self.model_type = "decoder-only"
                else:
                    print(f"Model {self.model_id} not supported.")
                    return

                model_downloaded = True
            except KeyboardInterrupt:
                print(
                    f"Model download interrupted. Retrying... (Attempt {retry_count + 1}/{max_retries})"
                )
                retry_count += 1
                # Clear GPU memory to avoid potential issues
                clear_memory()
                if retry_count == max_retries:
                    print("Max retry reached, skipping model download.")
                    return
            except Exception as e:
                print(f"An error occurred during model download: {e}")
                retry_count += 1
                # Clear GPU memory to avoid potential issues
                clear_memory()

                if retry_count == max_retries:
                    print("Max retry reached, skipping model download.")
                    return
        # Add padding token if it does not exist
        if self.tokenizer.pad_token is None:
            self.tokenizer.add_special_tokens({"pad_token": "[PAD]"})
            self.model.resize_token_embeddings(len(self.tokenizer))

        if not use_unsloth and not "unsloth" in self.model_id.lower():
            # Move model to device
            self.model.to(self.device)

        # Load Dataset (using dataset name from Hugging Face Hub)
        dataset = load_dataset(
            self.dataset_name, split="train", num_proc=self.config.get("dataset_num_proc", 2)
        )
        self.dataset = dataset.shuffle().select(
            range(self.config.get("dataset_size", 125))
        )

        print("\n")
        print("Observe finished.")
        return True


    @timeit
    def _orient(self):
        """
        Orients the agent by formatting the dataset and preparing training arguments.
        """
        print("\n")
        self.counter += 1
        print("Starting Orient ...")
        if self.dataset_name == "SetFit/mrpc":
            print("Dataset: SetFit/mrpc")
            preprocessing_function = self._preprocess_function_mrpc
        elif self.dataset_name == "b-mc2/sql-create-context":
            print("Dataset: b-mc2/sql-create-context")
            preprocessing_function = self._preprocess_function_sql_create_context
        elif self.dataset_name == "anthropic/hh-rlhf":
            print("Dataset: anthropic/hh-rlhf")
            preprocessing_function = self._preprocess_function_anthropic_hh_rlhf
        elif self.dataset_name == "imdb":
            print("Dataset: imdb")
            preprocessing_function = self._preprocess_function_imdb
        else:
            print(f"Dataset: {self.dataset_name} not supported.")
            return

        # Set the train/test split.
        test_size_percentage = self.config.get("test_split_percentage", 0.2)
        self.dataset = self.dataset.train_test_split(
            test_size=test_size_percentage
        )

        self.dataset = self.dataset.map(
            preprocessing_function,
            batched=True,
            remove_columns=self.dataset["train"].column_names,
        )

        # 3. Prepare Training Arguments
        # Import is_bfloat16_supported function.


        # Create TrainingArguments with the desired parameters
        training_args_config = self.config.get("training_args", {})
        self.training_args = TrainingArguments(
            output_dir=training_args_config.get("output_dir", "./output"),
            per_device_train_batch_size=training_args_config.get(
                "per_device_train_batch_size", 2
            ),
            gradient_accumulation_steps=training_args_config.get(
                "gradient_accumulation_steps", 4
            ),
            warmup_steps=training_args_config.get("warmup_steps", 5),
            max_steps=training_args_config.get("max_steps", 60),
            learning_rate=training_args_config.get("learning_rate", 2e-4),
            fp16=training_args_config.get("fp16", not is_bfloat16_supported()),
            bf16=training_args_config.get("bf16", is_bfloat16_supported()),
            logging_steps=training_args_config.get("logging_steps", 10),
            optim=training_args_config.get("optim", "adamw_8bit"),
            weight_decay=training_args_config.get("weight_decay", 0.01),
            lr_scheduler_type=training_args_config.get("lr_scheduler_type", "linear"),
            seed=training_args_config.get("seed", 3407),
            evaluation_strategy=training_args_config.get(
                "evaluation_strategy", "steps"
            ),  # we need this
            eval_steps=training_args_config.get("eval_steps", 20),
            save_strategy=training_args_config.get("save_strategy", "steps"),
            save_steps=training_args_config.get("save_steps", 20),
            report_to=training_args_config.get("report_to", "none"),
            remove_unused_columns=False # we need this
        )

        print("\n")
        print(f"Orient Dataset: {self.dataset}")

        print("\n")
        print("Orient finished.")
    @timeit
    def _decide(self):
        """
        Decides on the fine-tuning strategy, including LoRA configuration.
        """
        self.counter += 1
        print("\n")
        print("Starting Decide ...")
        clear_memory()
        # PEFT Configuration (LoRA)
        if self.config.get("lora"):
            self.model = prepare_model_for_kbit_training(self.model)
            if "bert" in self.model_id.lower():
                peft_config = LoraConfig(
                    lora_alpha=16,  # You can tune this.
                    lora_dropout=0.1,  # You can tune this.
                    r=64,  # You can tune this.
                    bias="none",
                    target_modules=["query", "key", "value", "dense"],  # Correct target modules for BERT
                    task_type="SEQ_CLS",  # correct task type
                )
            elif "mistral" in self.model_id.lower():
                peft_config = LoraConfig(
                    lora_alpha=128,
                    lora_dropout=0.05,
                    r=256,
                    bias="none",
                    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj", "down_proj"],
                    task_type="CAUSAL_LM",
                )
            elif "deepseek" in self.model_id.lower():
                peft_config = LoraConfig(
                    lora_alpha=128,
                    lora_dropout=0.1,
                    r=256,
                    bias="none",
                    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj", "down_proj"],
                    task_type="CAUSAL_LM",
                )
            elif "unsloth" in self.model_id.lower():
                peft_config = LoraConfig(
                    lora_alpha=128,
                    lora_dropout=0.05,
                    r=256,
                    bias="none",
                    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj", "down_proj"],
                    task_type="CAUSAL_LM",
                )
                print("\n")
                print(f"LORA: {peft_config}")

            else:
                print(f"Model {self.model_id} not supported.")
                return

            self.peft_config = peft_config
            self.model = get_peft_model(self.model, peft_config)

            self.model.print_trainable_parameters()


        print('\n')
        print("Decide finished.")

    @timeit
    def _act(self):
        """
        Acts by preprocessing the dataset and initializing the training loop.
        """
        self.counter += 1
        print("\n")
        print("Starting Act ...")
        clear_memory()

        try:
            if "train" not in self.dataset or "test" not in self.dataset:
                print(f"Missing train or test split for {self.dataset_name}")
                return

            print("Dataset preprocessed successfully.")
            print("\n")

            # Unsloth's Data Collator (Hypothetical)
            if self.config.get("use_unsloth", False) or "unsloth" in self.model_id.lower():
                print("Unsloth data collator used.")
                self.data_collator = DataCollatorWithPadding(tokenizer=self.tokenizer)
            else:
                # Hugging Face Data Collator
                self.data_collator = DataCollatorWithPadding(tokenizer=self.tokenizer)
                print("Hugging Face data collator used.")

            # Initialize Trainer
            print("Initializing Trainer...")
            loss_callback = LossLoggingCallback(self) # Create the callback
            metric_callback = MetricCallback(self)

            # Use the Trainer class instead of SFTTrainer
            self.trainer = Trainer(
                model=self.model,
                args=self.training_args,
                train_dataset=self.dataset["train"],
                eval_dataset=self.dataset["test"],
                data_collator=self.data_collator,
                #compute_metrics=self.compute_metrics,
                callbacks=[loss_callback, metric_callback]
            )

        except Exception as e:
            print(f"An error occurred in _act(): {e}")
            raise

        print("\n")
        print("Act finished.")


    import numpy as np

    from rouge_score import rouge_scorer

    from evaluate import load
    from rouge_score import rouge_scorer  # Import rouge-score
    import numpy as np
    import torch
    from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
    from sklearn.metrics import f1_score


    def calculate_bleu_score(self,hypothesis, references):
        """
        Calculates the BLEU score for a given hypothesis and list of references.

        Args:
            hypothesis (list of str): The candidate translation (a list of tokens).
            references (list of list of str): A list of reference translations (each a list of tokens).

        Returns:
            float: The BLEU score.
        """

        if not hypothesis or not references:
            return 0.0

        if any(not ref for ref in references):
            return 0.0

        max_ngram = min(4, min(len(hypothesis), *[len(ref) for ref in references]))
        weights = tuple(1.0 / max_ngram for _ in range(max_ngram))
        smoothing = SmoothingFunction().method4

        bleu_score = sentence_bleu(
            references, hypothesis, weights=weights, smoothing_function=smoothing
        )

        return bleu_score


    def calculate_f1_score(self, predictions, references):
        """
        Calculates the F1 score.
        """
        return f1_score(references, predictions, average='micro', zero_division=0)

    def compute_metrics(self, eval_pred):
        """
        Computes the BLEU, F1, ROUGE, and perplexity scores.

        Args:
            eval_pred (tuple): A tuple containing predictions and labels.

        Returns:
            dict: A dictionary containing the computed metrics.
        """
        predictions, labels = eval_pred
        metrics = {}  # Initialize an empty dictionary to store metrics

        # For sequence classification tasks, predictions are logits (like in BERT)
        if self.model_type == "encoder-only" and "bert" in self.model_id.lower():
            predictions = np.argmax(predictions, axis=1)
        # For decoder-only models, predictions need to be processed differently:
        elif self.model_type == "decoder-only":
            predictions = np.argmax(predictions, axis=2)  # Get the highest probability token for each position

        # Decode predictions and labels (if necessary)
        if self.model_type == "decoder-only":
            decoded_predictions = self.tokenizer.batch_decode(predictions, skip_special_tokens=True)
            # Replace -100 (ignore index) with pad_token_id in labels before decoding
            labels = np.where(labels != -100, labels, self.tokenizer.pad_token_id)
            decoded_labels = self.tokenizer.batch_decode(labels, skip_special_tokens=True)
        else:
            decoded_predictions = predictions  # Encoder-only models might not need decoding
            decoded_labels = labels

        # Extract references for BLEU calculation (nested list)
        references = [[label] for label in decoded_labels]

        # Calculate BLEU and F1 scores
        bleu_score = self.calculate_bleu_score(decoded_predictions, references)
        f1_score = self.calculate_f1_score(decoded_predictions, decoded_labels)
        metrics["bleu"] = bleu_score  # Add BLEU score to metrics
        metrics["f1"] = f1_score  # Add F1 score to metrics

        # Calculate ROUGE scores for decoder-only models
        if self.model_type == "decoder-only":
            scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
            all_scores = [scorer.score(label, pred) for label, pred in zip(decoded_labels, decoded_predictions)]

            # Calculate the average scores:
            rouge_metrics = {}
            for metric in ['rouge1', 'rouge2', 'rougeL']:
                precision = np.mean([score[metric].precision for score in all_scores])
                recall = np.mean([score[metric].recall for score in all_scores])
                fmeasure = np.mean([score[metric].fmeasure for score in all_scores])

                rouge_metrics[metric] = {
                    "precision": precision,
                    "recall": recall,
                    "fmeasure": fmeasure  # or "f1" if you prefer
                }
            metrics.update(rouge_metrics)  # Add rouge metrics to the main metrics dictionary

        # Calculate perplexity (for decoder-only models)
        if self.model_type == "decoder-only":
            try:
                with torch.no_grad():
                    # Prepare input for perplexity calculation
                    #inputs = self.tokenizer(decoded_labels, return_tensors="pt", padding=True).to(self.device)
                    # Pass inputs through the model to calculate loss
                    #outputs = self.model(**inputs, labels=inputs["input_ids"])
                    outputs = self.model(input_ids=torch.tensor(labels).to(self.device), labels=torch.tensor(labels).to(self.device))

                    loss = outputs.loss
                    # Calculate perplexity from loss
                    perplexity = torch.exp(torch.tensor(loss)).item()
                    metrics["perplexity"] = perplexity
            except Exception as e:
                print(f"Error in perplexity calculation: {e}")

        return metrics






    def compute_metricspoc2(self, eval_pred):
        """
        Computes the BLEU, F1, ROUGE, and Perplexity scores.

        Args:
            eval_pred (tuple): A tuple containing predictions and labels.

        Returns:
            dict: A dictionary containing the BLEU, F1, ROUGE, and perplexity scores.
        """
        predictions, labels = eval_pred
        metrics = {} #Initialize here

        # Handle decoder-only models:
        if self.model_type == "decoder-only":
            # Perplexity Calculation:
            with torch.no_grad():
                outputs = self.model(input_ids=torch.tensor(labels).to(self.device), labels=torch.tensor(labels).to(self.device))
                loss = outputs.loss
                perplexity = torch.exp(torch.tensor(loss)).item()
                #Add it to the dict
                metrics["perplexity"] = perplexity # Include perplexity if calculated

            predictions = np.argmax(predictions, axis=2) #argmax axis 2 for decoder only.
            decoded_predictions = self.tokenizer.batch_decode(predictions, skip_special_tokens=True)
            labels = np.where(labels != -100, labels, self.tokenizer.pad_token_id)
            decoded_labels = self.tokenizer.batch_decode(labels, skip_special_tokens=True)

        else: #encoder-decoder or other
            predictions = np.argmax(predictions, axis=1)
            decoded_predictions = predictions
            decoded_labels = labels
            # perplexity = None #Not needed, since you initialized metrics above.

        references = [[label] for label in decoded_labels]

        bleu_score = calculate_bleu_score(decoded_predictions, references)
        f1_score = calculate_f1_score(decoded_predictions,decoded_labels)

        rouge_scores = self.calculate_rouge(references, decoded_predictions)

        # Add to the dict
        metrics["bleu"] = bleu_score
        metrics["f1"] = f1_score

        #Add rouge
        for rouge_type, scores in rouge_scores.items():
                metrics[f"{rouge_type}_precision"] = scores['precision']
                metrics[f"{rouge_type}_recall"] = scores['recall']
                metrics[f"{rouge_type}_f1"] = scores['f1']


        # Return the updated metrics
        return metrics  # This line returns the dict


    def compute_metricspoc(self, eval_pred):
            """
            Computes the BLEU, F1, ROUGE, and perplexity metrics based on the task type.
            """
            predictions, labels = eval_pred

            # For sequence classification tasks (like MRPC), predictions are logits
            if self.model_type == "encoder-only" and "bert" in self.model_id.lower():
                predictions = np.argmax(predictions, axis=1)

            # Initialize an empty dictionary to store metrics
            metrics = {}

            # Decode predictions and labels
            if self.model_type == "decoder-only":
                decoded_predictions = self.tokenizer.batch_decode(predictions, skip_special_tokens=True)
                labels = np.where(labels != -100, labels, self.tokenizer.pad_token_id)
                decoded_labels = self.tokenizer.batch_decode(labels, skip_special_tokens=True)

                # Calculate BLEU and F1 scores
                references = [[label] for label in decoded_labels]
                bleu_score = calculate_bleu_score(decoded_predictions, references)
                f1_score = calculate_f1_score(decoded_predictions, decoded_labels)

                # Add BLEU and F1 to the metrics dictionary
                metrics["bleu"] = bleu_score
                metrics["f1"] = f1_score

                # Calculate ROUGE score
                rouge = load("rouge")
                results = rouge.compute(predictions=decoded_predictions, references=decoded_labels)
                metrics.update(results)  # Add ROUGE scores to the metrics dictionary

                # Calculate perplexity (with eval_loss check)
                try:
                    # Check if eval_loss is available in the log history
                    if self.trainer.state.log_history and "eval_loss" in self.trainer.state.log_history[-1]:
                        eval_loss = self.trainer.state.log_history[-1]["eval_loss"]
                        print(f"eval_loss found: {eval_loss}") # Debugging print statement

                        perplexity = torch.exp(torch.tensor(eval_loss)).item()
                        metrics["perplexity"] = perplexity
                    else:
                        print("eval_loss not found in logs for perplexity calculation. This is expected early in training or if evaluation has not occurred.")
                except Exception as e:
                    print(f"Error in perplexity calculation: {e}")
                    print(f"log_history: {self.trainer.state.log_history}")

                #Print all logs as Json file
                print(f"trainer.state.log_history: {json.dumps(self.trainer.state.log_history)}")


            else:  # For encoder-only models (like BERT)
                decoded_predictions = predictions
                decoded_labels = labels

                # Calculate BLEU and F1 scores (might not be relevant for all tasks)
                references = [[label] for label in decoded_labels]
                bleu_score = calculate_bleu_score(decoded_predictions, references)
                f1_score = calculate_f1_score(decoded_predictions, decoded_labels)

                # Add BLEU and F1 to the metrics dictionary
                metrics["bleu"] = bleu_score
                metrics["f1"] = f1_score

            return metrics

    !pip install evaluate -q
    from evaluate import load

    def compute_metricsgood(self, eval_pred):
        """
        Computes the BLEU and F1 scores.

        Args:
            eval_pred (tuple): A tuple containing predictions and labels.

        Returns:
            dict: A dictionary containing the BLEU and F1 scores.
        """
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)

        # Initialize an empty dictionary to store metrics
        #metrics = {}

        # Decode predictions and labels (if necessary)
        if self.model_type == "decoder-only":
          decoded_predictions = self.tokenizer.batch_decode(predictions, skip_special_tokens=True)
          labels = np.where(labels != -100, labels, self.tokenizer.pad_token_id)
          decoded_labels = self.tokenizer.batch_decode(labels, skip_special_tokens=True)
        else:
          decoded_predictions = predictions
          decoded_labels = labels

        # Extract references
        references = [[label] for label in decoded_labels]

        bleu_score = calculate_bleu_score(decoded_predictions, references)
        f1_score = calculate_f1_score(decoded_predictions,decoded_labels)

        # Calculate ROUGE score
        #rouge = load("rouge")
        #results = rouge.compute(predictions=decoded_predictions, references=decoded_labels)
        #metrics.update(results)  # Add ROUGE scores to the metrics dictionary


        # Add BLEU and F1 to the metrics dictionary
        #metrics["bleu"] = bleu_score
        #metrics["f1"] = f1_score

        return {"bleu": bleu_score, "f1": f1_score}
        #return metrics


    def on_train_loss(self, loss):
      """Callback to store training losses."""
      self.train_losses.append(loss)

    def on_eval_loss(self, loss):
        """Callback to store evaluation losses."""
        self.eval_losses.append(loss)
    @timeit
    def run(self):
        """
        Executes the OODA loop and fine-tunes the language model.
        """
        self.counter += 1
        print("\n")
        print("Starting Run ...")
        clear_memory()
        self.start_time = time.time()
        self._observe()
        if self.model is None:
            print("Model loading failed, skipping _orient, _decide and _act")
            return
        self._orient()
        self._decide()
        self._act()

        print("\n")
        print(f"Run Dataset: {self.dataset}")
        print("\n")

        if self.trainer is not None:
            try:
                # Train the model
                self.trainer.train()
                print("\n")
                print("Evaluation:")
                eval_results = self.evaluate()
                print("\n")
                print(eval_results)
                print("\n")

                # Create experiment_name
                # Create experiment_name (using triple quotes)

                experiment_name = f"""{self.model_id.replace('/', '-').replace("'", '')}_{self.dataset_name.replace('/', '-').replace("'", '')}"""
                # Save eval_results using write()

                import os
                import json  # Import json module
                current_directory = os.getcwd()
                %cd /content/
                results_file = os.path.join(current_directory, f"{experiment_name}_results.txt")
                with open(results_file, "w") as f:  # Open in write mode ("w")
                    json.dump(eval_results, f)  # Write eval_results as JSON
                    print(f"Saved evaluation results to: {results_file}")  # Add a print statement for confirmation

            except Exception as e:
                print(f"An error occurred during training or evaluation: {e}")
                raise
        else:
            print("Trainer is None. Skipping training and evaluation.")

        self.end_time = time.time()

        print("Run  finished.")
    @timeit
    def evaluate(self):
        """
        Evaluates the fine-tuned language model.
        """
        return self.trainer.evaluate()

    @timeit
    def _preprocess_function_mrpc(self, examples):
        """
        Preprocesses the data for the SetFit/mrpc dataset.
        Handles different model types and sequence lengths.
        """
        print("Preprocess Dataset: SetFit/mrpc")

        max_length = self.config.get("max_length", 128)  # Get max_length from config

        if self.model_type == "encoder-only":
            # BERT and other encoder-only models
            inputs = self.tokenizer(
                examples["text1"],
                examples["text2"],
                max_length=max_length,
                truncation=True,
                padding="max_length",
            )
            inputs["labels"] = examples["label"]
            return inputs
        elif self.model_type == "decoder-only":
             # Decoder-only models are not supported for the MRPC task.
            print("Decoder-only models are not supported for the MRPC task.")
            return {}
        else:
            raise ValueError(f"Unsupported model type: {self.model_type}")

    @timeit
    def _preprocess_function_sql_create_context(self, examples):
        """
        Preprocesses the data for the b-mc2/sql-create-context dataset.
        Handles different model types and sequence lengths.
        """
        print("Preprocess Dataset: b-mc2/sql-create-context")

        max_length = self.config.get("max_length", 1024)  # Get max_length from config

        if self.model_type == "decoder-only":
            # Mistral, DeepSeek, and other decoder-only models
            # Tokenize inputs and labels
            inputs = [f"### Question: {q} ### Context: {c}" for q, c in zip(examples["question"], examples["context"])]
            model_inputs = self.tokenizer(inputs, max_length=max_length, truncation=True, padding="max_length")
            # Tokenize labels
            labels_tokenized = self.tokenizer(examples["answer"], max_length=max_length, truncation=True, padding="max_length")
            # Assign labels to model_inputs
            model_inputs["labels"] = labels_tokenized["input_ids"]
            model_inputs["labels"] = [
                [(l if l != self.tokenizer.pad_token_id else -100) for l in label] for label in model_inputs["labels"]
            ]
        elif self.model_type == "encoder-only":
            # BERT and other encoder-only models
            # Tokenize inputs and labels
            inputs = [f"### Question: {q} ### Context: {c}" for q, c in zip(examples["question"], examples["context"])]
            model_inputs = self.tokenizer(inputs, max_length=max_length, truncation=True, padding="max_length")
            # Tokenize labels
            labels_tokenized = self.tokenizer(examples["answer"], max_length=max_length, truncation=True, padding="max_length")
            # Assign labels to model_inputs
            model_inputs["labels"] = labels_tokenized["input_ids"]
        else:
            raise ValueError(f"Unsupported model type: {self.model_type}")

        return model_inputs


    @timeit
    def _preprocess_function_anthropic_hh_rlhf(self, examples):
        """
        Preprocesses the data for the anthropic/hh-rlhf dataset.
        Handles different model types and sequence lengths.
        """
        print("Preprocess Dataset: anthropic/hh-rlhf")

        max_length = self.config.get("max_length", 1024)  # Get max_length from config

        if self.model_type == "decoder-only":
            # Mistral, DeepSeek, and other decoder-only models
            inputs = examples["chosen"]
            model_inputs = self.tokenizer(inputs, max_length=max_length, truncation=True, padding="max_length")
            # Tokenize labels
            labels_tokenized = self.tokenizer(examples["chosen"], max_length=max_length, truncation=True, padding="max_length")
            model_inputs["labels"] = labels_tokenized["input_ids"]
            model_inputs["labels"] = [
                [(l if l != self.tokenizer.pad_token_id else -100) for l in label] for label in model_inputs["labels"]
            ]
        elif self.model_type == "encoder-only":
            # BERT and other encoder-only models
            inputs = examples["chosen"]
            model_inputs = self.tokenizer(inputs, max_length=max_length, truncation=True, padding="max_length")
            # Tokenize labels
            labels_tokenized = self.tokenizer(examples["chosen"], max_length=max_length, truncation=True, padding="max_length")
            model_inputs["labels"] = labels_tokenized["input_ids"]
        else:
            raise ValueError(f"Unsupported model type: {self.model_type}")

        return model_inputs


    @timeit
    def _preprocess_function_imdb(self, examples):
        """
        Preprocesses the data for the imdb dataset.
        Handles different model types and sequence lengths.
        """
        print("Preprocess Dataset: imdb")

        max_length = self.config.get("max_length", 1024)  # Get max_length from config

        if self.model_type == "encoder-only":
             # BERT and other encoder-only models
            inputs = self.tokenizer(
                examples["text"],
                max_length=max_length,
                truncation=True,
                padding="max_length",
            )
            inputs["labels"] = examples["label"]
            return inputs
        elif self.model_type == "decoder-only":
            # Decoder-only models (Mistral, DeepSeek, etc.)
            model_inputs = self.tokenizer(
                examples["text"],
                max_length=max_length,
                truncation=True,
                padding="max_length",
            )
            # Copy input_ids to labels for causal LM training
            model_inputs["labels"] = model_inputs["input_ids"].copy()
            model_inputs["labels"] = [
                [(l if l != self.tokenizer.pad_token_id else -100) for l in label] for label in model_inputs["labels"]
            ]

            return model_inputs
        else:
            raise ValueError(f"Unsupported model type: {self.model_type}")

## Experiment Setup and Execution

In [ ]:
# Part 3: Experiment Setup and Execution
import os  # Import os module

class MetricCallback(TrainerCallback):
    """
    A callback class to add metrics to the trainer.
    """
    def __init__(self, agent):
        self.agent = agent

    def on_train_begin(self, args, state, control, model=None, **kwargs):
        # self.agent.trainer.compute_metrics = self.agent.compute_metrics # removed
        pass # removed

    def on_evaluate(self, args, state, control, model=None, **kwargs):
      """Callback to add metrics to self.trainer."""
      self.agent.trainer.compute_metrics = self.agent.compute_metrics # Added


class LossLoggingCallback(TrainerCallback):
    """Callback to log training and evaluation losses."""
    def __init__(self, agent):
        self.agent = agent

    def on_log(self, args, state, control, logs=None, **kwargs):
        """Logs the training loss at each log step."""
        if logs and "loss" in logs:
            self.agent.on_train_loss(logs["loss"])

    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        """Logs the evaluation loss at each evaluation step."""
        if metrics and "eval_loss" in metrics:
            self.agent.on_eval_loss(metrics["eval_loss"])



def create_rl_pairs():
    """
    Creates a list of all possible combinations of datasets, models,
    and configurations for RL experiments.
    """

    datasets = [
        #"SetFit/mrpc",
        #"b-mc2/sql-create-context",
        "anthropic/hh-rlhf",
        #"imdb",
    ]

    models = [

        "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
        #"unsloth/Meta-Llama-3.1-70B-bnb-4bit",
        #"bert-base-uncased",
        #"mistralai/Mistral-7B-v0.1",
        "deepseek-ai/deepseek-coder-1.3b-base",
    ]

    modelsfull = [
        "bert-base-uncased",
        "mistralai/Mistral-7B-v0.1",
        "deepseek-ai/deepseek-coder-1.3b-base",
        "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
        "unsloth/mistral-7b-bnb-4bit",
        "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
        "unsloth/llama-2-7b-bnb-4bit",
        "unsloth/llama-2-13b-bnb-4bit",
        "unsloth/codellama-34b-bnb-4bit",
        "unsloth/tinyllama-bnb-4bit",
        "unsloth/gemma-7b-bnb-4bit", # New Google 6 trillion tokens model 2.5x faster!
        "unsloth/gemma-2b-bnb-4bit",
        "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
        "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
        "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
        "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
        "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
        "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
        "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
        "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
        "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
        "unsloth/Phi-3-medium-4k-instruct",
        "unsloth/gemma-2-9b-bnb-4bit",
        "unsloth/gemma-2-27b-bnb-4bit",
    ]

    # Define different configs
    configs = [
        {
            "max_length": 32,
            "quantization": True,
            "use_unsloth": False,
            "lora": True,
            "dataset_size": 125,
            "dataset_num_proc": 2,
            "test_split_percentage": 0.2,
            "training_args": {
                "output_dir": "./output",
                "per_device_train_batch_size": 4,
                "gradient_accumulation_steps": 8,
                "warmup_steps": 5,
                "num_train_epochs": 5,
                "max_steps": 250,
                "learning_rate": 2e-4,
                "logging_steps": 10,
                "weight_decay": 0.01,
                "eval_steps": 20,
                "report_to": "none",
                "save_steps": 20,
                "evaluation_strategy":"steps",
                "eval_steps":20,
                "logging_strategy":"steps",
                "load_best_model_at_end":True,
                "metric_for_best_model":"eval_loss",

            },
        },
    ]

    rl_pairs = []
    for dataset, model, config in itertools.product(datasets, models, configs):
        rl_pairs.append((dataset, model, copy.deepcopy(config))) # Use copy.deepcopy()

    return rl_pairs

from tabulate import tabulate
import numpy as np
import time
from transformers import TrainingArguments, TrainerState, TrainerControl
import ast  # Import ast for literal_eval

def generate_report(
    rl_pairs, agents, training_args_list, state_list, control_list, output_file="experiment_report.txt", experiment_name=None
):
    """
    Generates a report for multiple RL experiments, including evaluation scores and training details.

    Args:
        rl_pairs (list): A list of tuples, each containing (dataset_name, model_id, config).
        agents (list): A list of FineTuningAgent objects corresponding to the experiments.
        training_args_list (list): A list of TrainingArguments objects for each experiment.
        state_list (list): A list of TrainerState objects for each experiment.
        control_list (list): A list of TrainerControl objects for each experiment.
        output_file (str): The name of the output file to save the report.
        experiment_name (str, optional): The base name for the experiment results file.
                                          If provided, it will be used to load the results.
                                          Defaults to None.
    """
    if not (
        len(rl_pairs)
        == len(agents)
        == len(training_args_list)
        == len(state_list)
        == len(control_list)
    ):
        raise ValueError("The number of rl_pairs, agents, training_args, state, and control must be the same.")

    report_data = []
    for (dataset_name, model_id, config), agent, training_args, state, control in zip(
        rl_pairs, agents, training_args_list, state_list, control_list
    ):

        # *** Load results from file ***
        if experiment_name:
            results_file = f"{experiment_name}_results.txt"  # Use provided experiment_name and .txt extension
        else:
            results_file = f"{dataset_name}_{model_id}_{agent.counter}_results.txt"  # Default format with .txt extension

        try:
            with open(results_file, "r") as f:  # Open in read mode ("r") for text files
                eval_results_str = f.read()  # Read the contents as a string
                # Try to parse eval_results_str as a Python literal (e.g., dictionary)
                try:
                    eval_results = ast.literal_eval(eval_results_str)
                except (SyntaxError, ValueError):
                    print(f"Error parsing eval_results_str for experiment: {results_file}")
                    eval_results = None
        except FileNotFoundError:
            print(f"Results file not found for experiment: {results_file}")
            eval_results = None  # Set to None if file not found

        # Collect the data
        elapsed_time = agent.end_time - agent.start_time if agent.start_time and agent.end_time else np.nan  # Handle potential errors

        train_losses = agent.train_losses
        eval_losses = agent.eval_losses

        if not train_losses:
            train_std = np.nan  # Use np.nan for no data
            min_train_loss = np.nan
            max_train_loss = np.nan
        else:
            train_std = np.std(train_losses)
            min_train_loss = np.min(train_losses)
            max_train_loss = np.max(train_losses)

        if not eval_losses:
            eval_std = np.nan
            min_eval_loss = np.nan
            max_eval_loss = np.nan
        else:
            eval_std = np.std(eval_losses)
            min_eval_loss = np.min(eval_losses)
            max_eval_loss = np.max(eval_losses)

        # *** Extract BLEU and F1 scores from eval_results ***
        if eval_results is not None:  # Check if eval_results were loaded successfully
            bleu_score = eval_results.get("eval_bleu", np.nan)  # Get BLEU score, default to NaN if not found
            f1_score = eval_results.get("eval_f1", np.nan)  # Get F1 score, default to NaN if not found
        else:
            bleu_score = np.nan  # Set to NaN if eval_results are None
            f1_score = np.nan

        # Check if training_args is None before accessing its attributes
        learning_rate = training_args.learning_rate if training_args is not None else np.nan
        batch_size = training_args.per_device_train_batch_size if training_args is not None else np.nan
        epochs = training_args.num_train_epochs if training_args is not None and hasattr(training_args, "num_train_epochs") else "n/a"

        report_data.append(
            [
                dataset_name,
                model_id,
                f"{elapsed_time:.2f} seconds",  # Format to 2 decimal places
                f"{train_std:.4f}",  # Format to 4 decimal places
                f"{eval_std:.4f}",  # Format to 4 decimal places
                f"{min_train_loss:.4f}",  # Format to 4 decimal places
                f"{max_train_loss:.4f}",  # Format to 4 decimal places
                f"{min_eval_loss:.4f}",  # Format to 4 decimal places
                f"{max_eval_loss:.4f}",  # Format to 4 decimal places
                f"{bleu_score:.4f}",  # Format to 4 decimal places  # Include BLEU score
                f"{f1_score:.4f}",  # Format to 4 decimal places  # Include F1 score
                f"{learning_rate:.4f}",  # Learning rate
                batch_size,  # Batch size
                epochs, # Epochs
                state.global_step if state else "n/a",  # Global steps
                state.epoch if state else "n/a",  # Epoch
                state.is_local_process_zero if state else "n/a",
                control.should_training_stop if control else "n/a",
                control.should_log if control else "n/a",
                control.should_save if control else "n/a",
            ]
        )

    headers = [
        "Dataset",
        "Model",
        "Elapsed Time",
        "Train Loss Std",
        "Eval Loss Std",
        "Min Train Loss",
        "Max Train Loss",
        "Min Eval Loss",
        "Max Eval Loss",
        "BLEU Score",  # Include header for BLEU Score
        "F1 Score",  # Include header for F1 Score
        "Learning Rate",
        "Batch Size",
        "Epochs",
        "Global Steps",
        "Epoch",
        "Is Local Process Zero",
        "Should Training Stop",
        "Should Log",
        "Should Save",
    ]


    # Format the report as a table
    report_table = tabulate(report_data, headers=headers, tablefmt="grid")

    # Print the report to the console
    print(report_table)

    # Save the report to a file
    with open(output_file, "w") as f:
        f.write(report_table)
        print(f"Report saved to {output_file}")

rl_pairs = create_rl_pairs()
# Run the experiment
import time

# Set CUDA_LAUNCH_BLOCKING before running the experiment loop
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # Enable synchronous CUDA operations
os.environ['TORCH_USE_CUDA_DSA'] = '1'  # Enable device-side assertions

agents = []
training_args_list = []
state_list = []
control_list = []
experiment_names = []

for dataset_name, model_id, config in rl_pairs:
    clear_memory()
    print("\n")
    print(f"Running experiment with:")
    print(f"- Dataset: {dataset_name}")
    print(f"- Model: {model_id}")
    print(f"- Config: {config}")
    print("\n")

    try:
        agent = FineTuningAgent(model_id, dataset_name, config)
        agents.append(agent) # Append the agent to the list immediately
        agent.start_time = time.time()
        agent.run()
        agent.end_time = time.time()
        # Collect training details after training
        if agent.trainer is not None:
          # Store experiment name and other relevant data
            experiment_name = f"""{model_id.replace('/', '-').replace("'", '')}_{dataset_name.replace('/', '-').replace("'", '')}"""
            experiment_names.append(experiment_name)
            # agents.append(agent) # Removed, agent has already been appended above
            training_args_list.append(agent.training_args)
            state_list.append(agent.trainer.state)
            control_list.append(agent.trainer.control)
        else:  # Append dummy values if training failed
            training_args_list.append(None)  # or a suitable placeholder
            state_list.append(None)
            control_list.append(None)

    except Exception as e:
        print(f"An error occurred during the experiment: {e}")
        agent.end_time = time.time()
        agent.start_time = time.time()
        training_args_list.append(None)  # or a suitable placeholder
        state_list.append(None)
        control_list.append(None)

# Call generate_report outside the loop, after all experiments are done
#generate_report(rl_pairs, agents, training_args_list, state_list, control_list, experiment_name=experiment_names)






Running experiment with:
- Dataset: anthropic/hh-rlhf
- Model: unsloth/mistral-7b-instruct-v0.3-bnb-4bit
- Config: {'max_length': 32, 'quantization': True, 'use_unsloth': False, 'lora': True, 'dataset_size': 125, 'dataset_num_proc': 2, 'test_split_percentage': 0.2, 'training_args': {'output_dir': './output', 'per_device_train_batch_size': 4, 'gradient_accumulation_steps': 8, 'warmup_steps': 5, 'num_train_epochs': 5, 'max_steps': 250, 'learning_rate': 0.0002, 'logging_steps': 10, 'weight_decay': 0.01, 'eval_steps': 20, 'report_to': 'none', 'save_steps': 20, 'evaluation_strategy': 'steps', 'logging_strategy': 'steps', 'load_best_model_at_end': True, 'metric_for_best_model': 'eval_loss'}}




Starting Run ...
Starting Observe ...
Mistral model detected. Using 4-bit quantization.
Decoder-only model detected.
Loading Decoder-only with Hugging Face


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

...

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Preprocess Dataset: anthropic/hh-rlhf
Function _preprocess_function_anthropic_hh_rlhf took 0.0457 seconds to execute


Map:   0%|          | 0/25 [00:00<?, ? examples/s]

Preprocess Dataset: anthropic/hh-rlhf
Function _preprocess_function_anthropic_hh_rlhf took 0.0127 seconds to execute


Orient Dataset: DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 25
    })
})


Orient finished.
Function _orient took 230.4316 seconds to execute


Starting Decide ...
trainable params: 671,088,640 || all params: 7,919,112,192 || trainable%: 8.4743


Decide finished.
Function _decide took 7.5291 seconds to execute


Starting Act ...
Dataset preprocessed successfully.


Unsloth data collator used.
Initializing Trainer...


Act finished.
Function _act took 0.4133 seconds to execute


Run Dataset: DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


## llm report

In [ ]:
import google.generativeai as genai
import numpy as np
from google.colab import userdata
import time
import json

# Used to securely store your API key
GOOGLE_API_KEY = userdata.get('GEMINI')  # Replace 'GEMINI' with your actual userdata variable name
genai.configure(api_key=GOOGLE_API_KEY)

from tabulate import tabulate
from transformers import TrainingArguments, TrainerState, TrainerControl

def generate_llm_report(
    rl_pairs,
    agents,
    training_args_list,
    state_list,
    control_list,
    output_file="experiment_report.txt",
    experiment_name=None,
    prompt="You are a helpful data science expert.\nPlease, make an additional analysis of this Fine-Tuning experiment report.",
):
    """
    Generates a report for multiple LLM experiments, including evaluation scores and training details,
    and provides an analysis using Google Gemini.

    Args:
        rl_pairs (list): A list of tuples, each containing (dataset_name, model_id, config).
        agents (list): A list of FineTuningAgent objects corresponding to the experiments.
        training_args_list (list): A list of TrainingArguments objects for each experiment.
        state_list (list): A list of TrainerState objects for each experiment.
        control_list (list): A list of TrainerControl objects for each experiment.
        output_file (str): The name of the output file to save the report.
        experiment_name (str, optional): The base name for the experiment results file.
                                        If provided, it will be used to load the results. Defaults to None.
        prompt (str, optional): The prompt to provide to Google Gemini for analysis.
                                Defaults to a generic data science expert prompt.
    """

    if not (
        len(rl_pairs)
        == len(agents)
        == len(training_args_list)
        == len(state_list)
        == len(control_list)
    ):
        raise ValueError(
            "The number of rl_pairs, agents, training_args, state, and control must be the same."
        )

    report_data = []  # Initialize report_data here

    for (
        (dataset_name, model_id, config),
        agent,
        training_args,
        state,
        control,
    ) in zip(rl_pairs, agents, training_args_list, state_list, control_list):
        # Get eval_results from the agent

        #print(f"Model ID: {model_id}")

        experiment_name = f"""{model_id.replace('/', '-').replace("'", '')}_{dataset_name.replace('/', '-').replace("'", '')}"""

        results_file = f"{experiment_name}_results.txt"
        #print(f"Results File: {results_file}")


        from pathlib import Path
        # Define the file path
        file_path = Path(results_file)

        if file_path.exists():
            #print("File exists!")
            #print(f"Results File: {results_file}")
            print
            #return
        else:
            #print("File does not exist.")
            continue


        training_args=agent.training_args

        experiment_name = f"""{model_id.replace('/', '-').replace("'", '')}_{dataset_name.replace('/', '-').replace("'", '')}"""
        #print(f"Experiment Name: {experiment_name}")

        results_file = f"{experiment_name}_results.txt"
        #print(f"Results File: {results_file}")

        print(training_args_list)


        # "eval_loss": 6.17133903503418, "eval_bleu": 0, "eval_f1": 0.0, "eval_runtime": 4.0188, "eval_samples_per_second": 6.221, "eval_steps_per_second": 0.995, "epoch": 8.64}

        try:
            with open(results_file, "r") as f:
                evaluation_results = json.load(f)
            bleu_score = evaluation_results.get("eval_bleu")
            f1_score = evaluation_results.get("eval_f1")
            print(f"BLEU Score: {bleu_score}, F1 Score: {f1_score}")
            #print(f"Eval Results: {evaluation_result}")

             # Accessing elements of the ROUGE score tuple (assuming it's a tuple with precision, recall, fmeasure)
            rouge1_precision = evaluation_results.get('eval_rouge1', {}).get('precision')
            rouge1_recall = evaluation_results.get('eval_rouge1', {}).get('recall')
            rouge1_fmeasure = evaluation_results.get('eval_rouge1', {}).get('fmeasure')
            print(f"ROUGE-1 Precision: {rouge1_precision}, Recall: {rouge1_recall}, F-measure: {rouge1_fmeasure}")

            # Accessing elements of the ROUGE score tuple (assuming it's a tuple with precision, recall, fmeasure)


            rouge2_precision = evaluation_results.get('eval_rouge2', {}).get('precision')
            rouge2_recall = evaluation_results.get('eval_rouge2', {}).get('recall')
            rouge2_fmeasure = evaluation_results.get('eval_rouge2', {}).get('fmeasure')


            # Accessing elements of the ROUGE score tuple (assuming it's a tuple with precision, recall, fmeasure)
            rougeL_precision = evaluation_results.get('eval_rougeL', {}).get('precision')
            rougeL_recall = evaluation_results.get('eval_rougeL', {}).get('recall')
            rougeL_fmeasure = evaluation_results.get('eval_rougeL', {}).get('fmeasure')


           # {"eval_loss": 3.720398187637329, "eval_bleu": 0, "eval_f1": 0.0, "eval_rouge1": {"precision": 0.396081703619663, "recall": 0.40836230306837384, "fmeasure": 0.40190438521178246}, "eval_rouge2": {"precision": 0.07832895449176241, "recall": 0.08075765617251687, "fmeasure": 0.07946492337001675}, "eval_rougeL": {"precision": 0.3623965364533813, "recall": 0.37335556072231263, "fmeasure": 0.3675868060434595},
            #"eval_perplexity": 45.677459716796875, "eval_runtime": 1.1153, "eval_samples_per_second": 22.415, "eval_steps_per_second": 3.586, "epoch": 62.64}

            perplexity = evaluation_results.get("eval_perplexity", "N/A")
            print(f"Perplexity: {perplexity}")

        except (FileNotFoundError, json.JSONDecodeError) as e:
            print(f"Error loading results: {e}")
            bleu_score = None
            f1_score = None




        # Collect the data
        elapsed_time = (
            agent.end_time - agent.start_time
            if agent.start_time and agent.end_time
            else np.nan
        )  # Handle potential errors
        train_losses = agent.train_losses
        eval_losses = agent.eval_losses

        if not train_losses:
            train_std = np.nan  # Use np.nan for no data
            min_train_loss = np.nan
            max_train_loss = np.nan
        else:
            print(f"Train Losses: {train_losses}")
            train_loss = np.mean(train_losses)
            train_loss_std = np.std(train_losses)
            min_train_loss = np.min(train_losses)
            max_train_loss = np.max(train_losses)

        if not eval_losses:
            eval_std = np.nan
            min_eval_loss = np.nan
            max_eval_loss = np.nan
        else:
            print(f"Eval Losses: {eval_losses}")
            eval_loss = np.mean(eval_losses)
            eval_loss_std = np.std(eval_losses)
            min_eval_loss = np.min(eval_losses)
            max_eval_loss = np.max(eval_losses)

        # Check if training_args is None before accessing its attributes
        learning_rate = training_args.learning_rate if training_args is not None else np.nan
        batch_size = training_args.per_device_train_batch_size if training_args is not None else np.nan
        epochs = training_args.num_train_epochs if training_args is not None and hasattr(training_args, "num_train_epochs") else "n/a"
        print(f"Learning Rate: {learning_rate}")
        print(f"Batch Size: {batch_size}")
        print(f"Epochs: {epochs}")

        report_data.append(
            [
                dataset_name,
                model_id,
                f"{elapsed_time:.2f} seconds",  # Format to 2 decimal places
                f"{train_loss:.4f}" if train_loss is not None else "N/A",  # Handle None case for train_loss
                f"{eval_loss:.4f}" if eval_loss is not None else "N/A",  # Handle None case for eval_loss

                f"{train_loss_std:.4f}" if train_loss_std is not None else "N/A",  # Handle None case for train_loss_std
                f"{eval_loss_std:.4f}" if eval_loss_std is not None else "N/A",  # Handle None case for eval_std

                f"{min_train_loss:.4f}" if min_train_loss is not None else "N/A",  # Handle None case for min_train_loss
                f"{max_train_loss:.4f}" if max_train_loss is not None else "N/A",  # Handle None case for max_train_loss

                f"{min_eval_loss:.4f}" if min_eval_loss is not None else "N/A",  # Handle None case for min_eval_loss
                f"{max_eval_loss:.4f}" if max_eval_loss is not None else "N/A",  # Handle None case for max_eval_loss




                f"{bleu_score:.4f}" if bleu_score is not None else "N/A",  # Handle None case for bleu_score
                f"{f1_score:.4f}" if f1_score is not None else "N/A",  # Handle None case for f1_score

                f"{rouge1_precision:.4f}" if rouge1_precision is not None else "N/A",  # Handle None case for rouge1_precision
                f"{rouge1_recall:.4f}" if rouge1_recall is not None else "N/A",  # Handle None case for rouge1_recall
                f"{rouge1_fmeasure:.4f}" if rouge1_fmeasure is not None else "N/A",  # Handle None case for rouge1_fmeasure

                f"{rouge2_precision:.4f}" if rouge2_precision is not None else "N/A",  # Handle None case for rouge2_precision
                f"{rouge2_recall:.4f}" if rouge2_recall is not None else "N/A",  # Handle None case for rouge2_recall
                f"{rouge2_fmeasure:.4f}" if rouge2_fmeasure is not None else "N/A",  # Handle None case for rouge2_fmeasure

                f"{rougeL_precision:.4f}" if rougeL_precision is not None else "N/A",  # Handle None case for rougeL_precision
                f"{rougeL_recall:.4f}" if rougeL_recall is not None else "N/A",  # Handle None case for rougeL_recall
                f"{rougeL_fmeasure:.4f}" if rougeL_fmeasure is not None else "N/A",  # Handle None case for rougeL_fmeasure

                f"{perplexity:.4f}" if perplexity is not None else "N/A",  # Handle None case for perplexity

                f"{learning_rate:.4f}" if learning_rate is not None else "N/A",  # Handle None case for learning_rate
                f"{batch_size}" if batch_size is not None else "N/A",  # Handle None case for batch_size
                f"{epochs}" if epochs is not None else "N/A",  # Handle None case for epochs

                #f"{state.global_step}" if state is not None else "N/A",  # Handle None case for global_step
                #f"{state.epoch}" if state is not None else "N/A",  # Handle None case for epoch






                #f"{learning_rate:.4f}",  # Learning rate
                #batch_size,  # Batch size
                #epochs,  # Epochs
                #state.global_step if state else "n/a",  # Global steps
                #state.epoch if state else "n/a",  # Epoch
                #state.is_local_process_zero if state else "n/a",
                #control.should_training_stop if control else "n/a",
                #control.should_log if control else "n/a",
               # control.should_save if control else "n/a",
            ]
        )

    # Generate the report table
    headers = [
        "Dataset",
        "Model",
        "Elapsed Time",
        "Train Loss (Average)",
        "Eval Loss (Average)",
        "Train Loss (Std)",
        "Eval Loss (Std)",
        "Min Train Loss",
        "Max Train Loss",
        "Min Eval Loss",
        "Max Eval Loss",
        "BLEU Score",
        "F1 Score",
        "ROUGE-1 Precision",
        "ROUGE-1 Recall",
        "ROUGE-1 F-measure",
        "ROUGE-2 Precision",
        "ROUGE-2 Recall",
        "ROUGE-2 F-measure",
        "ROUGE-L Precision",
        "ROUGE-L Recall",
        "ROUGE-L F-measure",
        "Perplexity",
        "Learning Rate",
        "Batch Size",
        "Epochs",
        "Global Steps",
        "Epoch",
        "is_local_process_zero",
        "should_training_stop",
        "should_log",
        "should_save",
    ]



    report_table = tabulate(report_data, headers=headers, tablefmt="grid")

    # Save the report to a file
    with open(output_file, "w") as f:
        f.write(report_table)

    print(report_table)

    # LLM Analysis using Google Gemini
    model_name = "gemini-1.5-pro"  # Replace with desired model
    model = genai.GenerativeModel(model_name)
    response = model.generate_content(prompt + "\n\n" + report_table)
    llm_analysis = response.text

    print("\n\n## LLM Analysis:\n")
    print(llm_analysis)

    #return llm_analysis



In [ ]:
# Generate the LLM report and send to Gemini
prompt = """
You are a helpful data science expert.
Please, make an additional analysis of this Fine-Tuning experiment report.
"""
# Initialize training_args_list, state_list, control_list with empty lists
training_args_list = [None] * len(rl_pairs)
state_list = [None] * len(rl_pairs)
control_list = [None] * len(rl_pairs)

generate_llm_report(rl_pairs, agents, training_args_list, state_list, control_list, prompt=prompt)

[None]
BLEU Score: 0, F1 Score: 0.0
ROUGE-1 Precision: 0.396081703619663, Recall: 0.40836230306837384, F-measure: 0.40190438521178246
Perplexity: 45.677459716796875
Train Losses: [2.252, 1.3595, 0.5566, 0.2696, 0.2246, 0.2156, 0.1887, 0.1934, 0.1851, 0.1813, 0.1768, 0.1711, 0.1755, 0.168, 0.1719, 0.1718, 0.166, 0.1625, 0.1708, 0.1765, 0.1591, 0.1641, 0.1611, 0.1642, 0.1569]
Eval Losses: [2.2944350242614746, 2.939926862716675, 3.2473931312561035, 3.44758677482605, 3.519420862197876, 3.6173152923583984, 3.673027753829956, 3.651576280593872, 3.6940383911132812, 3.679342746734619, 3.6730434894561768, 3.710360050201416, 3.720398187637329]
Learning Rate: 0.0002
Batch Size: 4
Epochs: 3.0
+-------------------+--------------------------------------+-----------------+------------------------+-----------------------+--------------------+-------------------+------------------+------------------+-----------------+-----------------+--------------+------------+---------------------+------------------